# Julia 

In [1]:
import Pkg
Pkg.add("Flux")
Pkg.add("Plots")
Pkg.add("NetCDF")
Pkg.add("TSVD")
Pkg.add("IterTools")
Pkg.add("Statistics")
Pkg.add("Compat")
Pkg.add("LinearAlgebra")
Pkg.add("Glob")
using Flux
using Flux: train!
using Plots
using TSVD
using IterTools
using IterTools: ncycle 
using Statistics
using LinearAlgebra
using Compat
using Glob
using NetCDF

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [2]:
training_files = glob("usurf_gris_g1800m_v5_RAGIS_id_*_1980-1-1_2020-1-1_YM.nc")
nf = length(training_files)
d = ncread(training_files[1], "usurf")
nx, ny, nt = size(d)
x = ncread(training_files[1], "x")
y = ncread(training_files[1], "y")

Data = zeros(nf * nt, nx*ny)
for (k, training_file) in enumerate(training_files)
    println(training_file)
    d = ncread(training_file, "usurf")
    data = transpose(reshape(d, ny * nx, nt))
    Data[(k - 1 ) * nt + 1:k * nt, :] = data
end


obs_file ="aerodem_1978_1987_wgs84_g1800m.nc"
obs = ncread(obs_file, "surface_altitude")
nx_obs, ny_obs = size(obs)

R = reshape(obs, 1, nx * ny);
I = findall(x->x!=-9999, R);

usurf_gris_g1800m_v5_RAGIS_id_0_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_10_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_11_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_12_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_13_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_14_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_15_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_16_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_17_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_18_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_19_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_1_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_20_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_21_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_22_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_23_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_24_1980-1-1_2020-1-1_YM.nc
usurf_gris_g1800m_v5_RAGIS_id_25_

In [3]:
q = 10

Data_mean = mean(Data, dims=1)
Data_scaled = Data .- Data_mean
U, S, V = tsvd(Data_scaled, q)
println("dim V = ", size(V))
println("dim S = ", size(S))
println("dim U = ", size(U))
println("dim R = ", size(R))

SV = transpose(V * diagm(S))

println("SV= ",size(SV))

x_train = transpose(SV[repeat(I, 1, q)])
_, n_obs = size(x_train)
y_train = reshape(R[I], 1, n_obs)



V= (1337600, 10)
S= (10,)
U= (1120, 10)
R= (1, 1337600)
SV= (10, 1337600)


1×176918 Matrix{Float32}:
 24.5947  -2.37123  202.58  -3.77586  …  -8.43288  -13.9553  -7.86059

In [ ]:
for K in [0 1e5 1e6 1e7 1e8]
    println(K)
    model = Dense(q, 1)
    ps = Flux.params(model)
    penalty() = K * mean(abs2, model.weight)
    loss(x, y) = Flux.Losses.mse(model(x), y) + penalty()
    opt = Adam(0.1)


    loss_0 = loss(x_train, y_train)
    println("Initial loss: $loss_0")

    data = [(x_train, y_train)]
    n_epochs = 100

    train_loader = Flux.DataLoader((x_train, y_train), batchsize = 128, shuffle = true)

    Flux.train!(loss, ps, ncycle(train_loader, n_epochs), opt)

    loss_e = loss(x_train, y_train)
    println("Final loss: $loss_e")

    M =  reshape(model.weight * SV + Data_mean, nx, ny)
    M[findall(x->x<0, M)] .= 0

    Ms = copy(reshape(M, nx, ny, 1))
    heatmap(x, y, transpose(M))

    filename = "dem_q_$(q)_K_$(K).nc"
    varname  = "usurf"
    attribs  = Dict("units"   => "m",
                    "data_min" => 0.0)

    nccreate(filename, varname, "x", x, "y", y, "time", 1, atts=attribs)
    ncwrite(Ms, filename, varname)
end

0.0
Initial loss: 535137.848512128
Final loss: 274503.15228713304
100000.0
Initial loss: 595094.1244531599
